In [1]:
import pandas as pd
import random
import numpy as np
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import unidecode
import re
from spacy.cli import download
import spacy
import nltk
import contractions
from sklearn.model_selection import train_test_split
import gensim

spacy.cli.download("en_core_web_sm")
nltk.download('stopwords')

from nltk.corpus import stopwords


2026-01-18 23:33:37.812220: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-18 23:33:38.432366: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-18 23:33:41.786100: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/tguyot/PersonalCode/spam-classifier/.venv/lib/python3.12/site-packages/keras/src/expo

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.1 MB/s  0:00:02 eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package stopwords to /home/tguyot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Import cleaned preprocess csv
df = pd.read_csv('cleaned_spam.csv').drop(columns=['Unnamed: 0'])
df

,is_spam,sms
0,1,go jurong point crazy available bugis n great ...
1,1,ok lar joking wif oni
2,0,free entry wkly comp win fa cup final tkts st ...
3,1,dun say early hor c already say
4,1,nah I think go usf live around though
...,...,...
5556,0,nd time try contact win aPS pound prize claim ...
5557,1,I b go esplanade fr home
5558,1,pity mood soany suggestion
5559,1,guy bitching I act like I interested buy somet...


In [3]:
def get_corpus(df):
    return ' '.join(' '.join(df.sms.tolist()).split())

def get_unique_words(df):
    corpus = get_corpus(df)
    unique = set()
    output = []
    for word in corpus.split():
        if word not in unique:
            unique.add(word)
            output.append(word)
    return output

def count_unique(df):
    return len(get_unique_words(df))



count_unique(df)

get_unique_words(df)[:10] 

['go',
 'jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'n',
 'great',
 'world',
 'la']

In [4]:
# Test with gensim first


X_train, X_test, y_train, y_test = train_test_split(df['sms'].apply(lambda x: x.split()),
                                                    df['is_spam'],
                                                    test_size=0.2,
                                                    random_state=42)
w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=300,
                                   window=5,
                                   min_count=1)

# Test word vectors
w2v_model.wv.most_similar('phone')

[('text', 0.9998831152915955),
 ('free', 0.9998794198036194),
 ('txt', 0.999877393245697),
 ('reply', 0.9998714327812195),
 ('get', 0.999868631362915),
 ('send', 0.999865710735321),
 ('message', 0.9998558759689331),
 ('take', 0.9998549818992615),
 ('go', 0.9998549818992615),
 ('call', 0.999854564666748)]

In [35]:
# Continuous bag of words

def generate_cbows(text, window_size):
    """Generate Continuous Bag of Words (CBOW) pairs from the given text. 
    text: preprocessed sequence of text"""
    # Tokenize the text
    words = text.split()

    # Create CBOW pairs with a given window size
    cbows = []
    for i, target_word in enumerate(words):
        context_words = words[max(0, i - window_size):i] + words[i + 1:i + window_size + 1]
        if len(context_words) == window_size * 2:
            cbows.append((context_words, target_word))
    return cbows

# Create cbows
cbows = generate_cbows(get_corpus(df), window_size=5)

# Display the results
for context_words, target_word in cbows[:10]:
    print(f'Context Words: {context_words}, Target Word: {target_word}')

Context Words: ['go', 'jurong', 'point', 'crazy', 'available', 'n', 'great', 'world', 'la', 'e'], Target Word: bugis
Context Words: ['jurong', 'point', 'crazy', 'available', 'bugis', 'great', 'world', 'la', 'e', 'buffet'], Target Word: n
Context Words: ['point', 'crazy', 'available', 'bugis', 'n', 'world', 'la', 'e', 'buffet', 'cine'], Target Word: great
Context Words: ['crazy', 'available', 'bugis', 'n', 'great', 'la', 'e', 'buffet', 'cine', 'get'], Target Word: world
Context Words: ['available', 'bugis', 'n', 'great', 'world', 'e', 'buffet', 'cine', 'get', 'amore'], Target Word: la
Context Words: ['bugis', 'n', 'great', 'world', 'la', 'buffet', 'cine', 'get', 'amore', 'wat'], Target Word: e
Context Words: ['n', 'great', 'world', 'la', 'e', 'cine', 'get', 'amore', 'wat', 'ok'], Target Word: buffet
Context Words: ['great', 'world', 'la', 'e', 'buffet', 'get', 'amore', 'wat', 'ok', 'lar'], Target Word: cine
Context Words: ['world', 'la', 'e', 'buffet', 'cine', 'amore', 'wat', 'ok', 'lar

In [36]:
def one_hot_encoding(word: str, unique_words: list):
    try:
        assert word in set(unique_words)
    except AssertionError:
        print(f"Word '{word}' not found in unique_words.")
        return None
    vector = np.zeros(len(unique_words))
    index = unique_words.index(word)
    vector[index] = 1
    return tf.convert_to_tensor(vector)


# Create one hot encodings

unique_words = list(get_unique_words(df))
one_hot_encodings = {
    word: one_hot_encoding(word, unique_words) for word in unique_words
}



In [37]:
# Convert CBOW pairs to vector pairs
cbow_vector_pairs = [([one_hot_encodings[word] for word in context_words], one_hot_encodings[target_word]) for context_words, target_word in cbows]

# Sum the context vectors to get a single context vector
cbow_vector_pairs = [(tf.reduce_sum(tf.stack(context_vectors), axis=0), target_vector) for context_vectors, target_vector in cbow_vector_pairs]

In [38]:
cbow_vector_pairs[:5]  # Display first 5 CBOW vector pairs

[(<tf.Tensor: shape=(7339,), dtype=float64, numpy=array([1., 1., 1., ..., 0., 0., 0.], shape=(7339,))>,
  <tf.Tensor: shape=(7339,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.], shape=(7339,))>),
 (<tf.Tensor: shape=(7339,), dtype=float64, numpy=array([0., 1., 1., ..., 0., 0., 0.], shape=(7339,))>,
  <tf.Tensor: shape=(7339,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.], shape=(7339,))>),
 (<tf.Tensor: shape=(7339,), dtype=float64, numpy=array([0., 0., 1., ..., 0., 0., 0.], shape=(7339,))>,
  <tf.Tensor: shape=(7339,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.], shape=(7339,))>),
 (<tf.Tensor: shape=(7339,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.], shape=(7339,))>,
  <tf.Tensor: shape=(7339,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.], shape=(7339,))>),
 (<tf.Tensor: shape=(7339,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.], shape=(7339,))>,
  <tf.Tensor: shape=(7339,), dtype=float64, numpy=array([0.,

In [39]:
class NaiveWord2Vec(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.weight1 = tf.keras.layers.Dense(embedding_dim, input_shape=(vocab_size,), activation='linear', name='embedding_layer')
        self.weight2 = tf.keras.layers.Dense(vocab_size, input_shape=(embedding_dim,), activation='softmax')
    
    def call(self, inputs):
        x = self.weight1(inputs)
        x = self.weight2(x)
        return x
    

    

In [ ]:
VOCAB_SIZE = len(unique_words)
VECTOR_DIM = 300

model = NaiveWord2Vec(VOCAB_SIZE, VECTOR_DIM)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'] # this is just word2vec
)

model.fit(
    x=tf.stack([pair[0] for pair in cbow_vector_pairs]),
    y=tf.stack([pair[1] for pair in cbow_vector_pairs]),
    epochs=10,
    batch_size=128, 
    validation_split=0.1
)


Epoch 1/10
353/353 ━━━━━━━━━━━━━━━━━━━━ 22s 55ms/step - accuracy: 0.0749 - loss: 7.4140 - val_accuracy: 0.0839 - val_loss: 7.1054
Epoch 2/10
353/353 ━━━━━━━━━━━━━━━━━━━━ 19s 54ms/step - accuracy: 0.0870 - loss: 6.4840 - val_accuracy: 0.0987 - val_loss: 6.8435
Epoch 3/10
353/353 ━━━━━━━━━━━━━━━━━━━━ 19s 53ms/step - accuracy: 0.1101 - loss: 5.7006 - val_accuracy: 0.1075 - val_loss: 6.6974
Epoch 4/10
203/353 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.1542 - loss: 4.8738

In [ ]:
model.get_layer('embedding_layer').get_weights()

[array([[-0.2397558 ,  0.11211057,  0.04273067, ..., -0.17481793,
          0.02447324,  0.19437271],
        [ 0.01840967, -0.05884297, -0.02350952, ...,  0.09782509,
          0.2138359 ,  0.09622125],
        [ 0.24445544, -0.27639169, -0.15053558, ...,  0.18468523,
         -0.23221882,  0.15553747],
        ...,
        [-0.26258704, -0.1626276 , -0.09193443, ..., -0.18885404,
         -0.12884265,  0.18520987],
        [-0.26023152,  0.02098822, -0.066674  , ..., -0.11843918,
          0.09807865, -0.04264724],
        [-0.08031567, -0.03805484, -0.03699608, ...,  0.05741283,
         -0.00482738, -0.01303974]], shape=(7339, 300), dtype=float32),
 array([ 8.27972144e-02, -7.63747171e-02,  4.96348627e-02,  4.90374155e-02,
        -6.90865740e-02, -2.99599897e-02, -1.32833719e-02,  4.51385416e-02,
         7.77688846e-02,  3.01445033e-02,  5.25377020e-02, -1.37138650e-01,
        -6.39462918e-02, -4.57772948e-02,  8.13158415e-03, -1.09567314e-01,
         1.51452180e-02,  7.7691912

In [ ]:
# Get word embeddings
embeddings = model.get_layer('embedding_layer').get_weights()[0]
word_embeddings = {word: embeddings[idx] for idx, word in enumerate(unique_words)
}
word_embeddings['phone']  # Example embedding for the word 'phone'

array([-3.15498084e-01, -6.93529770e-02, -3.11394818e-02,  4.49059606e-02,
        6.07568435e-02,  1.79082140e-01, -2.33514830e-01,  4.60990258e-02,
       -5.88371530e-02, -3.88602257e-01, -2.97492981e-01, -9.47781876e-02,
        3.94366771e-01,  2.80486494e-01,  2.31879577e-01,  1.38568416e-01,
       -3.26728761e-01, -1.42800305e-02,  8.98759216e-02,  2.35805407e-01,
        4.07174349e-01, -2.40666598e-01,  3.34708810e-01,  4.08957526e-02,
        6.88036010e-02, -2.35001482e-02,  2.65577231e-02, -3.41521442e-01,
       -5.77514470e-02,  1.74865305e-01, -1.30803496e-01, -3.35410804e-01,
        2.98039645e-01, -8.37064460e-02, -3.77371013e-01, -1.14328325e-01,
        4.68625277e-02, -1.31423324e-01,  1.32083148e-01,  8.74082670e-02,
        1.67630821e-01, -3.58226113e-02, -2.37804562e-01,  9.93765797e-03,
       -8.21259841e-02, -2.11912200e-01,  8.19912739e-03, -2.35730708e-01,
        1.07919194e-01,  2.39611566e-01, -2.06310749e-01,  2.53899582e-02,
       -4.78892058e-01, -

In [ ]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

In [ ]:
cosine_similarity(word_embeddings['phone'], word_embeddings['nokia'])

np.float32(0.123276554)

In [ ]:
cosine_similarity(word_embeddings['phone'], word_embeddings['mom'])

np.float32(-0.0053015216)

In [ ]:
cosine_similarity(word_embeddings['mom'], word_embeddings['dad'])

np.float32(0.1789304)